<a href="https://colab.research.google.com/github/kfirraiby/test-repo/blob/master/multiclassification_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ovo multicalssification  -  text sentiment analysis 

- In this work I will present a classic machine learning model pipeline for  text analysis.

- The purpose of the model is to estimate rating that a person will give to a movie based on the review he wrote.

- The model is a logistic regression with a lasso penalty, which receives a lemmatized (with part of speech taken into account) n-gram (1,3)  tf-idf as features 

***note:***:i this file there is a ***full pipeline***  for preprocessing, feature desine, train,test and prediction only for the final modle.
there is no  exploratory data analysis or model comparison. all this was done in previous work and this file is the final result  





In [0]:
!pip install imblearn

In [0]:
import pandas as pd 
import numpy as np
import re

import imblearn
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler

import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import  preprocessing
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

from collections import Counter

import warnings
warnings.filterwarnings("ignore")


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#**Reading, uploading and filtering**

Because of the computation power, I will filter the data for classes 1 (bad), 3 (ok), and 5 (good).
From this I will sample  about 2,000,000 observations while maintaining the imbalanced problem.  
<br>
The data is raw, meaning that we need to look in the raw text where the Review is and where the ranking is per customer
<br>
example:
<br>
<br>
product/productId: B00006HAXW
<br>
review/userId: A1RSDE90N6RSZF
<br>
review/profileName: Joseph M. Kotow
<br>
review/helpfulness: 9/9
<br>
review/score: 5.0
<br>
review/time: 1042502400
<br>
review/summary: Pittsburgh - Home of the OLDIES
<br>
review/text: I have all of the doo wop DVD's and this one is as good or better than the
1st ones. Remember once these performers are gone, we'll never get to see them again.
Rhino did an excellent job and if you like or love doo wop and Rock n Roll you'll LOVE
this DVD !!
<br>
<br>

you can find the details on the raw data in https://snap.stanford.edu/data/web-Movies.html


 






In [0]:
# Connecting to Google Drive where the raw data is
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Search and save the ranking from raw text
file = open("/content/drive/My Drive/Colab Notebooks/movies.txt", "r",errors="ignore")
scores = []
for line in file: 
  if line[:13] == "review/score:":
    scores.append(float(line[14:]))

#Search and save the review
file =  open("/content/drive/My Drive/Colab Notebooks/movies.txt", "r",errors="ignore") 
reviews = []
for line in file: 
  if line[:12] == "review/text:":
    reviews.append(line[12:])
    
file.close()

#saving in a pandas DataFrame
df = pd.DataFrame({'score': scores ,'text': reviews})
del scores
del reviews

# sampling and maintaining the imbalanced problem.  
data_full = df.sample(n = 2000000, weights="score",replace=False,random_state=42)[df["score"].isin([1,3,5])]
del df

In [0]:
print(data_full.score.value_counts(normalize = True))
print(data_full.shape)

5.0    0.870197
3.0    0.100803
1.0    0.028999
Name: score, dtype: float64
(1523559, 2)


In [0]:
#example
print(data_full["text"].loc[2968573])
print(data_full["score"].loc[2968573])

 In a true sense, Clint Eastwood & writer David Peoples have created the first Anti-western in this tale of violence, vengeance and half-truths. Eastwood plays the anti-hero William Munny, once a vicious killer, now reformed eking out an existence for his two motherless children by pig farming. To save his children from this hapless fate he agrees to go on one last mercenary killing to avenge a prostitute whose face, eyes, ears and teets were cut up by some clients or so he's heard. But the truth of it is less than the fiction as the whore's story, like so many tales from the old west, is continually embellished. Urged on by the raging Schofield Kid, a haplessly poor-sighted lad who only dreams of the notoriety his killing might bring, Munny enlists the aid of Morgan Freeman, his old partner. Munny so rusty he can no longer aim well and so long out of the saddle he can't even stay on his horse, he heads out with the other two avenging or fallen angels to Big Whiskey to hunt down the ki

#**text preration class**

-   ***costume stop words*** 
-   ***lower case and remove special characters, numbers, punctuation and others***
-  ***lemmilemmatize by part of speech***













In [0]:
# extract POS tags in dict
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].lower()
    tag_dict = {"j": wordnet.ADJ,
                "n": wordnet.NOUN,
                "v": wordnet.VERB,
                "r": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

# i implemnt the text preperation in a class that we can use it in a pipeline later on 

class TextPrepTransformer(BaseEstimator, TransformerMixin):
  def fit(self, X,y=None):
    return self
  def transform(self,X):
      remove = ["don","don't","no","not","too","very","isn","isn't","wasn","wasn't","weren","weren't","won","won't","wouldn","wouldn't","shouldn't","shouldn","mustn't","mustn"]
      S_words = list(stopwords.words("english"))
      word_tokenizer = WhitespaceTokenizer()
      lemmatizer = WordNetLemmatizer()
#remove costume stopwords
      new_list = []
      for e in S_words:
          if e not in remove:
              new_list.append(e)
      new_list.append("www")
      new_list.append("com")
      new_list.append("quot")
      new_list.append("br")
      S_words = new_list

  #clean text
      X = X.apply(lambda x: " ".join(x.lower() for x in x.split()))
      X = X.apply(lambda x: " ".join(x.strip() for x in x.split()))
      X = X.replace(r"http\S+", "")
      X = X.replace(r"www.\S+", "")
      X = X.str.replace('[^A-Za-z]+', ' ')
      X = X.str.replace(r"[^\w\s]", '')
      X = X.apply(lambda x: ' '.join([word for word in x.split() if word not in (S_words)]))
      X = X.astype(str)
    
  #lemmitize by part of speech and tokenize
      for i in X.index:
        X[i] = [lemmatizer.lemmatize(w,get_wordnet_pos(w)) for w in word_tokenizer.tokenize(X[i])]
  
      return X




    
   
    

      





#***imbalance problem*** 

For the problem of imbalance I wanted to use more complex techniques.
For each model I wanted to use an undersampling method that takes into account the density.
<br>
note that oversampling here will create a lot of noisy data because  of high sparsity and  is not necessary sience we have  fair number of observations in the small class

when i tested the undersampling method that takes into account the density (NearMiss algo) it was ***after*** processing the data and extracting the features, but **unfortunately**, there was not enough computing power.
 So I just had to undesampled it randomly before processing.



In [0]:
RS = RandomUnderSampler("not minority")
X_sm, y_sm = RS.fit_sample(pd.DataFrame(data_full["text"]),data_full["score"].astype("category"))
data_full = pd.merge(pd.DataFrame(X_sm),pd.DataFrame(y_sm),left_index=True,right_index=True)
data_full = data_full.rename(columns={"0_x": "text", "0_y": "score"})
del X_sm, y_sm

In [0]:
data_full.score.value_counts()

5.0    44182
3.0    44182
1.0    44182
Name: score, dtype: int64

# train and test in a pipeline

This model is the chosen model after hyperparameter tuning, comparing different algorithms and comparing the different combinations of the NLP techniques above

tested:
- naive baye, logistic regression and  random forest 
-  stemming, lemmatization and lemmatization + POS
- countvectorizer, 1-3 gram, tf-idf (and combined)
- random under sampling and near miss

all of the above where tested with a simple train test split and chosen by acuuracy and auc score




In [0]:

models =  ["53","51","31"]

for idx, mod in enumerate(models):
  #filter data for the model
  data = data_full.loc[(data_full["score"]==float(mod[0])) | (data_full["score"]==float(mod[1]))]
  print("model:", mod)
 
  X_model, X_test_final, y_model, y_test_final = train_test_split(data["text"], data["score"], test_size = 0.1, random_state=42,shuffle = True)
  
  if mod == "53":
    
    model_53 = Pipeline(steps = [ 
                           ("text prep",TextPrepTransformer()),
                           ("cvec",CountVectorizer(min_df = 0.005,max_df=0.9,ngram_range=(1,3), tokenizer=lambda doc: doc, lowercase=False)),
                           ("TD", TfidfTransformer()),
                           ("estimator53", LogisticRegression(penalty="l1"))
                          ])
    model_53.fit(X_model,y_model)
    y_pred_test = model_53.predict(X_test_final)
    print("test results:",classification_report(y_test_final,y_pred_test))
  
  elif mod == "51":
    model_51 = Pipeline(steps = [ 
                           ("text prep",TextPrepTransformer()),
                           ("cvec",CountVectorizer(min_df = 0.005,max_df=0.9,ngram_range=(1,3), tokenizer=lambda doc: doc, lowercase=False)),
                           ("TD", TfidfTransformer()),
                           ("estimator51", LogisticRegression(penalty="l1"))
                            ])
    model_51.fit(X_model,y_model)
    y_pred_test = model_51.predict(X_test_final)
    print("test results:",classification_report(y_test_final,y_pred_test))
    
  elif mod == "31":
    model_31 = Pipeline(steps = [ 
                           ("text prep",TextPrepTransformer()),
                           ("cvec",CountVectorizer(min_df = 0.005,max_df=0.9,ngram_range=(1,3), tokenizer=lambda doc: doc, lowercase=False)),
                           ("TD", TfidfTransformer()),
                           ("estimator31", LogisticRegression(penalty="l1"))
                          ])
    model_31.fit(X_model,y_model)
    y_pred_test = model_31.predict(X_test_final)
    print("test results:",classification_report(y_test_final,y_pred_test))
  

    
  
 
  

model: 53
test results:               precision    recall  f1-score   support

         3.0       0.83      0.83      0.83      4470
         5.0       0.83      0.83      0.83      4367

    accuracy                           0.83      8837
   macro avg       0.83      0.83      0.83      8837
weighted avg       0.83      0.83      0.83      8837

model: 51
test results:               precision    recall  f1-score   support

         1.0       0.92      0.91      0.92      4470
         5.0       0.91      0.92      0.92      4367

    accuracy                           0.92      8837
   macro avg       0.92      0.92      0.92      8837
weighted avg       0.92      0.92      0.92      8837

model: 31
test results:               precision    recall  f1-score   support

         1.0       0.84      0.83      0.84      4470
         3.0       0.83      0.84      0.84      4367

    accuracy                           0.84      8837
   macro avg       0.84      0.84      0.84      8837
we

#***prediction***

In [0]:
#majority vote

def vote(text):
  result = []
  for mod in [model_53,model_51,model_31]:
    result.append(float(mod.predict(pd.Series(text))))
  d = Counter(result)
  inverse = [(value, key) for key, value in d.items()]    
  return print("review:",text,"\n","estimated score:" ,max(inverse)[1],"\n","--------------------")
    
    

In [0]:
# prediction
vote("i love that movie")
vote("i hate this movie")
vote("the movie was nice, not more then that")
vote("i love steven spielberg but this was terrible")
vote("I realy  looked forward to this film and it was exactly what I expected")
vote("I realy looked forward to this movie and i was a bit disappointed")
vote("i hate the actor but it was amazing")
vote("i hate the actor and the movie")




review: i love that movie 
 estimated score: 5.0 
 --------------------
review: i hate this movie 
 estimated score: 1.0 
 --------------------
review: the movie was nice, not more then that 
 estimated score: 3.0 
 --------------------
review: i love steven spielberg but this was terrible 
 estimated score: 1.0 
 --------------------
review: I realy  looked forward to this film and it was exactly what I expected 
 estimated score: 5.0 
 --------------------
review: I realy looked forward to this movie and i was a bit disappointed 
 estimated score: 3.0 
 --------------------
review: i hate the actor but it was amazing 
 estimated score: 5.0 
 --------------------
review: i hate the actor and the movie 
 estimated score: 1.0 
 --------------------
